In [4]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/train/', 
    classes=['other','samsung'],
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary', 
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/test/',
    classes=['other', 'samsung'],
    target_size=(150,150),
    batch_size=100,
    class_mode='binary', 
)

Found 18752 images belonging to 2 classes.
Found 8331 images belonging to 2 classes.


In [8]:
from keras import models 
from keras import layers 

# VGG16 합성곱 기반 층 만들기 

from keras.applications import vgg16
from keras import optimizers

# 합성곱 기반 층
conv_base = vgg16.VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

# 모델 정의
model = models.Sequential() 
model.add(conv_base)# 합성곱 기반 층
model.add(layers.Flatten()) # 합성곱 기반 층 출력 1차원 텐서(벡터로)변형
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# 합성곱 기반 층 가중치 동결 
conv_base.trainable = False
print(f'{len(model.trainable_weights)}') 


# 모델 컴파일 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=1,
    validation_data = validation_generator, 
    validation_steps=83)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-02-25 01:26:46.323850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-25 01:26:46.324820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


8


2022-02-25 01:26:47.789636: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-25 01:26:48.160317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - ETA: 0s - loss: 0.5420 - accuracy: 0.7505

2022-02-25 01:30:09.877093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - 274s 1s/step - loss: 0.5420 - accuracy: 0.7505 - val_loss: 0.1880 - val_accuracy: 0.9976


In [18]:
from keras.preprocessing import image 
from keras.applications.vgg16 import preprocess_input

img_path = '/Users/kibeomkim/Desktop/samsung6.jpg'

img = image.load_img(img_path, target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

result = model.predict(x) ; print(result)

[[1.]]


In [12]:
import os 
valid_list = os.listdir('/Users/kibeomkim/Desktop/openlogo_split/test/samsung') 

def galaxy(name) : 
    img = image.load_img(f'/Users/kibeomkim/Desktop/openlogo_split/test/samsung/{name}', target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x) 

result = [galaxy(image) for image in valid_list]

In [15]:
print(len(result))
print(len([x for x in result if x > 0.5]))

19
11


In [16]:
result

[array([[1.]], dtype=float32),
 array([[0.08696464]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.01046942]], dtype=float32),
 array([[0.4492039]], dtype=float32),
 array([[0.00357371]], dtype=float32),
 array([[2.4573163e-10]], dtype=float32),
 array([[0.04657805]], dtype=float32),
 array([[7.665175e-08]], dtype=float32),
 array([[0.9389554]], dtype=float32),
 array([[0.57686275]], dtype=float32),
 array([[0.02162392]], dtype=float32),
 array([[0.99998844]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.660773]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.9849288]], dtype=float32)]

In [39]:
# 인스타그램 이미지 투입해보자. 

valid_list = os.listdir('/Users/kibeomkim/Desktop/인스타그램_크롤링_결과/#galaxys21_2000') 

def galaxy(name) : 
    img = image.load_img(f'/Users/kibeomkim/Desktop/인스타그램_크롤링_결과/#galaxys21_2000/{name}', target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x) 

result = [galaxy(image) for image in valid_list]

In [41]:
len([x for x in result if x > 0.5]) # 2022 02 25 모델 수정 필요. 

1220

In [43]:
1220/len(result)

0.5211448099102948